In [ ]:
from datetime import date
import datetime
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import pandas as pd

In [2]:
# project cash flow inputs and outputs for a given time period based on a cash flow stream definition

class StreamProjection:
    def __init__(self,
                 ps_start_date, # date string
                 ps_end_date,   # date string
                 stream):
        
        # convert time period start and end dates strings into date types
        p_start_date=parse(ps_start_date).date()
        p_end_date=parse(ps_end_date).date()
    
        # define the time step to be used to genrate dates for ouccrences of cash flow event baed on stream definition
        delta = relativedelta(**stream['stream_step']) # use dictionary as mapping argument

        # intialise cash flow event date iterator
        i_date=parse(stream['stream_start_date']).date()
                 
        # intialise list of cash flow event dates to be generated   
        p_str_dates=[]

        # generate cash flow event dates until end of time period
        while i_date <= p_end_date:
            
            # select only cash flow event whos dates are within the  time period and within the stream defintion
            if (i_date >= p_start_date) & (i_date<= parse(stream['stream_end_date']).date()):
                
                # append generated date to list for cash flow event dates
                p_str_dates.append({'occurence_date':i_date,
                                'occurence_value':stream['occurence_value'],
                                'stream_name':stream['stream_name']})
                
            # increment date iteratior bby previously defined step amount
            i_date += delta
        
        # create class attributes with list of cash flow event dates gnerated
        self.stream_dates=p_str_dates
        
        return

In [ ]:
# for list of stream definitions and defined time period create dataframe aggregate by year with cash flow stream as columns
class Projection:
    def __init__(self,
                 ps_start_date,
                 ps_end_date,
                 streams,
                 grouper):
        
        # created list of projected streams
        stream_projection=[]

        # for each stream create list of cash flow events and append it to a list for all events
        for i in streams:
            stream_projection=stream_projection+StreamProjection(ps_start_date,ps_end_date,i).stream_dates

        # convert list of cash flow event into a dataframe of cash flow events for each stream 
        df=pd.DataFrame(stream_projection)
        
        # convert dataframe of cash flow event into dataframe with date as index and the columns for each stream containing their value
        df2=df.pivot(index="occurence_date", columns="stream_name", values="occurence_value")
        
        # convert date index to datetime with period of month
        df2.index = pd.to_datetime(df2.index).to_period('M')
        
        # aggregate dataframe to year level
        self.df=df2.groupby(pd.Grouper(freq=grouper)).sum()
        
        return